In [ ]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,017 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,313 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:14 http:

In [ ]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-17 05:07:56--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.44MB/s    in 0.2s    

2023-03-17 05:07:57 (4.44 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [ ]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [ ]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
watches_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep= ('\t'), header=True, inferSchema=True)

# Show DataFrame
watches_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [ ]:
#check columns in dataframe
column_names = watches_data_df.columns
column_names

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [ ]:
# Get the number of rows in the DataFrame.
watches_data_df.count()


960872

# Transform the Data

## Create the "review_id_table".

In [ ]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = watches_data_df.select(['review_id', 'customer_id','product_id','product_parent',to_date('review_date','yyyy-mm-dd').alias('review_date')])
review_id_df.show() # will only show 20 rows


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3O9SGZBVQBV76|    3653882|B00FALQ1ZC|     937001370| 2015-01-31|
| RKH8BNC3L5DLF|   14661224|B00D3RGO20|     484010722| 2015-01-31|
|R2HLE8WKZSU3NL|   27324930|B00DKYC7TK|     361166390| 2015-01-31|
|R31U3UH5AZ42LL|    7211452|B000EQS1JW|     958035625| 2015-01-31|
|R2SV659OUJ945Y|   12733322|B00A6GFD7S|     765328221| 2015-01-31|
| RA51CP8TR5A2L|    6576411|B00EYSOSE8|     230493695| 2015-01-31|
| RB2Q7DLDN6TH6|   11811565|B00WM0QA3M|     549298279| 2015-01-31|
|R2RHFJV0UYBK3Y|   49401598|B00A4EYBR0|     844009113| 2015-01-31|
|R2Z6JOQ94LFHEP|   45925069|B00MAMPGGE|     263720892| 2015-01-31|
| RX27XIIWY5JPB|   44751341|B004LBPB7Q|     124278407| 2015-01-31|
|R15C7QEZT0LGZN|    9962330|B00KGTVGKS|      28017857| 2015-01-31|
|R361XSS37V0NCZ|   16097204|B0039UT5OU|     685450910| 2015-01

In [ ]:
review_id_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

## Create the "products" Table

In [ ]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = watches_data_df.select(['product_id', 'product_title'])
products_df.show() # will only show 20 rows

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00FALQ1ZC|Invicta Women's 1...|
|B00D3RGO20|Kenneth Cole New ...|
|B00DKYC7TK|Ritche 22mm Black...|
|B000EQS1JW|Citizen Men's BM8...|
|B00A6GFD7S|Orient ER27009B M...|
|B00EYSOSE8|Casio Men's GW-94...|
|B00WM0QA3M|Fossil Women's ES...|
|B00A4EYBR0|INFANTRY Mens Nig...|
|B00MAMPGGE|G-Shock Men's Gre...|
|B004LBPB7Q|Heiden Quad Watch...|
|B00KGTVGKS|Fossil Women's ES...|
|B0039UT5OU|Casio General Men...|
|B00MPF0XJQ|2Tone Gold Silver...|
|B003P1OHHS|Bulova Men's 98B1...|
|B00R70YEOE|Casio - G-Shock -...|
|B000FVE3BG|Invicta Men's 332...|
|B008X6JB12|Seiko Women's SUT...|
|B0040UOFPW|Anne Klein Women'...|
|B00UR2R5UY|Guess U13630G1 Me...|
|B00HFF57L0|Nixon Men's Geo V...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
#Drop duplicates
products_df = products_df.drop_duplicates()

## Create the "customers" Table

In [ ]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = watches_data_df.groupby('customer_id').count()
customers_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|    1567510|    1|
|   19502021|    1|
|   12819130|    1|
|   35329257|    2|
|     108460|    1|
|    5453476|    1|
|   29913055|    1|
|   30717305|    1|
|    1570030|    1|
|   19032020|    1|
|   44178035|    1|
|   26079415|    2|
|   14230926|    1|
|   43478048|    2|
|   43694941|    1|
|   12318815|    3|
|   13731855|    1|
|     740134|    1|
|   41956754|    1|
|   20324070|    3|
+-----------+-----+
only showing top 20 rows



## Create the "vine_table".

In [ ]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df= watches_data_df.select(['review_id', 'star_rating','helpful_votes','total_votes','vine'])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3O9SGZBVQBV76|          5|            0|          0|   N|
| RKH8BNC3L5DLF|          5|            0|          0|   N|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|
|R31U3UH5AZ42LL|          5|            0|          0|   N|
|R2SV659OUJ945Y|          4|            0|          0|   N|
| RA51CP8TR5A2L|          5|            0|          0|   N|
| RB2Q7DLDN6TH6|          5|            1|          1|   N|
|R2RHFJV0UYBK3Y|          1|            1|          5|   N|
|R2Z6JOQ94LFHEP|          5|            1|          2|   N|
| RX27XIIWY5JPB|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



# Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://postgres.ccxnnnes366j.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", 
          "password": "******", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine', mode=mode, properties=config)